In [25]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
from torch.optim import Adam
from sklearn.metrics import r2_score

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class HeatmapCNN(nn.Module):
    def __init__(self):
        super(HeatmapCNN, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(in_features=32 * 14 * 20, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 32 * 14 * 20)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x


In [27]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from scipy.special import softmax


class HeatMapDataset(Dataset):
    def __init__(self, file_name):
        self.df = pd.read_csv(file_name)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        heatmap_str = self.df.iloc[idx, 10][1:-1]

        data = np.fromstring(heatmap_str, sep=',') 
        
        height = 56
        width = 80
        channels = 3
        
        #-------------------------------------------------

        reshape_old = np.array(data).reshape(channels, height, width)
        
        reshape_old[0] = softmax(reshape_old[0])
        reshape_old[1] = softmax(reshape_old[1])
        reshape_old[2] = softmax(reshape_old[2])
        
        #-------------------------------------------------
        
        reshape_new = reshape_old.transpose((1,2,0))

        output = reshape_new.copy()
        
        r =  reshape_new[:,:,0]
        g =  reshape_new[:,:,1]
        b = reshape_new[:,:,2]

        r_min = r.min()
        g_min = g.min()
        b_min = b.min()

        r_max = r.max()
        g_max = g.max()
        b_max = b.max()

        output[:,:,0] = (reshape_new[:,:,2]-b_min)/(b_max-b_min)
        output[:,:,1] = (reshape_new[:,:,1]-g_min)/(g_max-g_min)
        output[:,:,2] = (reshape_new[:,:,0]-r_min)/(r_max-r_min)

        output = np.where(output > 0.1, output, np.zeros_like(output))
        
        reshape = output.transpose(2, 0, 1)
        
        
        targets = self.df.iloc[idx, 6:8].values.astype(np.float32)
        
        
        
        return torch.tensor(reshape, dtype=torch.float), torch.tensor(targets, dtype=torch.float)

In [36]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        i = 0
        for inputs, targets in train_loader:
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            i += 1
        
        print(i)
        epoch_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                
        val_loss /= len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')

In [32]:
dataset = HeatMapDataset('/Users/abhianna/Downloads/corn_env6_y=-0.2_yaw=0_rosbag.csv')
valset  = HeatMapDataset('/Users/abhianna/Downloads/corn_env5_y=-0.2_yaw=0_rosbag.csv')


In [39]:
train_loader = DataLoader(dataset, batch_size=10, shuffle=True)
for inputs, targets in train_loader:
    print("hello")

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


In [37]:
train_loader = DataLoader(dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(valset, batch_size=10) 

model = HeatmapCNN()
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, val_loader, criterion, optimizer)

11
Epoch 1/25, Loss: nan, Validation Loss: nan
11
Epoch 2/25, Loss: nan, Validation Loss: nan
11
Epoch 3/25, Loss: nan, Validation Loss: nan
11
Epoch 4/25, Loss: nan, Validation Loss: nan
11
Epoch 5/25, Loss: nan, Validation Loss: nan
11
Epoch 6/25, Loss: nan, Validation Loss: nan
11
Epoch 7/25, Loss: nan, Validation Loss: nan
11
Epoch 8/25, Loss: nan, Validation Loss: nan
11
Epoch 9/25, Loss: nan, Validation Loss: nan
11
Epoch 10/25, Loss: nan, Validation Loss: nan
11
Epoch 11/25, Loss: nan, Validation Loss: nan
11
Epoch 12/25, Loss: nan, Validation Loss: nan
11
Epoch 13/25, Loss: nan, Validation Loss: nan
11
Epoch 14/25, Loss: nan, Validation Loss: nan
11
Epoch 15/25, Loss: nan, Validation Loss: nan
11
Epoch 16/25, Loss: nan, Validation Loss: nan
11
Epoch 17/25, Loss: nan, Validation Loss: nan
11
Epoch 18/25, Loss: nan, Validation Loss: nan
11
Epoch 19/25, Loss: nan, Validation Loss: nan
11
Epoch 20/25, Loss: nan, Validation Loss: nan
11
Epoch 21/25, Loss: nan, Validation Loss: nan
1